In [1]:
import numpy as np
import random
import load_data
from sample_code_submission.model import model
import re

In [2]:
limit_files_per_town = 5

Cellule suivante à remplacer par une fonction : img, gt = load_data(path)

In [3]:
img = load_data.load_images_in_path("./split_data/img_split_RGB_filtered", limit_files_per_town)

town_name_list = load_data.get_town_name_list(img)

def dict_to_sorted_list(img_dict):
    return sorted(img_dict.items(), key=lambda t: t[0])

sorted_img = dict_to_sorted_list(img)

gt = load_data.load_images_in_path("./split_data/gt_split_filtered", limit_files_per_town)
sorted_gt = dict_to_sorted_list(gt)

def make_train_valid_sets(sorted_train, sorted_gt, town_name_list):
    ratio = 0.8
    train_img_list = []
    train_gt_list = []
    valid_img_list = []
    valid_gt_list = []
    for (n1,img),(n2,gt) in zip(sorted_train, sorted_gt):
        if n1 != n2:
            print("not sorted !")
        if random.random() < ratio:
            train_img_list.append((n1,img))
            train_gt_list.append((n2,gt))
        else:
            valid_img_list.append((n1,img))
            valid_gt_list.append((n2,gt))
    return {"img":train_img_list, "gt":train_gt_list}, {"img":valid_img_list,"gt":valid_gt_list}

train, valid = make_train_valid_sets(sorted_img, sorted_gt, town_name_list)

def img_sorted_list_to_numpy(sorted_img_list):
    return np.asarray([(np.moveaxis(np.asarray(img).astype(float), -1, 0)/255.)[np.newaxis,:,:,:] for _,img in sorted_img_list])

train_np = img_sorted_list_to_numpy(train["img"])

def gt_sorted_list_to_numpy(sorted_gt_img):
    return np.asarray([np.where(np.asarray(gt).flatten() > 122, 1, 0) for _,gt in sorted_gt_img])

gt_np = gt_sorted_list_to_numpy(train["gt"])

valid_img_np = img_sorted_list_to_numpy(valid["img"])
valid_gt_np = gt_sorted_list_to_numpy(valid["gt"])

In [4]:
print(valid_gt_np.shape)
print(valid_img_np.shape)
print(train_np.shape)
print(gt_np.shape)

(5, 1000000)
(5, 1, 3, 1000, 1000)
(14, 1, 3, 1000, 1000)
(14, 1000000)


In [5]:
m = model(verbose=True)

In [6]:
m.fit(train_np, gt_np)

Epoch 0, image 0


KeyboardInterrupt: 

In [6]:
pred = m.predict(valid_img_np)

In [9]:
print(pred)
print(pred.shape)
print(valid_img_np.shape[0], valid_img_np.shape[3] * valid_img_np.shape[4])

[[0.64711154 0.64690655 0.65070134 ... 0.64243585 0.6486125  0.64258927]
 [0.64753675 0.64642948 0.6481697  ... 0.64528537 0.65627539 0.64720386]
 [0.65467811 0.65100664 0.65654171 ... 0.64095926 0.6396054  0.64061582]
 [0.64984667 0.64760727 0.65094239 ... 0.64852899 0.64952713 0.64240193]
 [0.64756948 0.6456849  0.64796287 ... 0.64477777 0.65136892 0.64358878]]
(5, 1000000)
5 1000000
